# Demos: Lecture 11

## Demo 1: `qml.specs`

In [ ]:
from functools import partial
from pprint import pprint

import pennylane as qml

from lecture11_helpers import decompose

This is our implementation of Grover's algorithm from last class:

In [ ]:
n_bits = 3
n_work_wires = 2
special_string = '110'

dev = qml.device('default.qubit', wires=n_bits+1+n_work_wires)

def hadamard_transform(wires=None):
    for wire in wires:
        qml.Hadamard(wires=wire)

def oracle():
    qml.MultiControlledX(
        control_wires=range(n_bits), 
        wires=n_bits, 
        control_values=special_string,
        work_wires=range(n_bits+1, n_bits+1+n_work_wires)
    )
    
def diffusion():
    hadamard_transform(wires=range(n_bits))
    qml.MultiControlledX(
        control_wires=range(n_bits), 
        wires=n_bits, 
        control_values='0'*n_bits,
        work_wires=range(n_bits+1, n_bits+1+n_work_wires)
    )
    hadamard_transform(wires=range(n_bits))


In [ ]:
def grover(num_steps=1):
    qml.PauliX(wires=n_bits)
    
    hadamard_transform(wires=range(n_bits+1))
        
    for _ in range(num_steps):
        oracle()
        diffusion()
        
    return qml.probs(wires=range(n_bits))    

In [ ]:
grover_qnode = qml.QNode(grover, dev)
print(qml.draw(grover_qnode)(num_steps=2))

In [ ]:
pprint(qml.specs(grover_qnode)(num_steps=2)['resources'])

## Demo 2: Decomposing multi-qubit operations

In [ ]:
grover_qnode_l1 = qml.QNode(decompose(grover, level=1), dev)

In [ ]:
print(qml.draw(grover_qnode_l1)(num_steps=2))

In [ ]:
pprint(qml.specs(grover_qnode_l1)(num_steps=2)['resources'])

In [ ]:
grover_qnode_l2 = qml.QNode(decompose(grover, level=2), dev)
print(qml.draw(grover_qnode_l2)(num_steps=2))

In [ ]:
pprint(qml.specs(grover_qnode_l2)(num_steps=2)['resources'])

## Demo 3: apply custom optimization pipeline

In [ ]:
pipeline = [
    qml.transforms.cancel_inverses,
    partial(qml.transforms.commute_controlled, direction="left"),
    partial(qml.transforms.merge_rotations, atol=1e-6),
    qml.transforms.cancel_inverses,
    partial(qml.transforms.commute_controlled, direction="left"),
    partial(qml.transforms.merge_rotations, atol=1e-6),
]

In [ ]:
@qml.qnode(dev)
@qml.compile(pipeline=pipeline, num_passes=2)
@decompose(level=2)
def grover(num_steps=1):
    qml.PauliX(wires=n_bits)
    
    hadamard_transform(wires=range(n_bits+1))
        
    for _ in range(num_steps):
        oracle()
        diffusion()
        
    return qml.probs(wires=range(n_bits))    

In [ ]:
print(qml.draw(grover)())

In [ ]:
pprint(qml.specs(grover)()['resources'])